Importing the Libararies


In [4]:
# Install required packages with specified versions
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets huggingface-hub



In [13]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import notebook_login
import re


notebook_login ()

# Changing existing data set to our dataset

In [6]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("keivalya/MedQuad-MedicalQnADataset")

# Shuffle the dataset and slice it
dataset = ds['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data
def transform_conversation(example):
    # Extract patient question and doctor answer
    patient_text = example.get('Question', '').strip()
    doctor_text = example.get('Answer', '').strip()

    # Format the conversation using the specified template
    reformatted_text = f"<s>[INST] Patient: {patient_text} [/INST] Doctor: {doctor_text} </s>"

    return {'text': reformatted_text}

# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

# Print a sample to verify
print(transformed_dataset[0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'qtype': 'inheritance', 'Question': 'Is D-bifunctional protein deficiency inherited ?', 'Answer': 'This condition is inherited in an autosomal recessive pattern, which means both copies of the gene in each cell have mutations. The parents of an individual with an autosomal recessive condition each carry one copy of the mutated gene, but they typically do not show signs and symptoms of the condition.', 'text': '<s>[INST] Patient: Is D-bifunctional protein deficiency inherited ? [/INST] Doctor: This condition is inherited in an autosomal recessive pattern, which means both copies of the gene in each cell have mutations. The parents of an individual with an autosomal recessive condition each carry one copy of the mutated gene, but they typically do not show signs and symptoms of the condition. </s>'}


In [14]:
transformed_dataset.push_to_hub("medicalllama3-1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rakesh-04/medicalllama3-1k/commit/94fc7063fa32236be2149c55ec1caae76ba7bcd1', commit_message='Upload dataset', commit_description='', oid='94fc7063fa32236be2149c55ec1caae76ba7bcd1', pr_url=None, pr_revision=None, pr_num=None)